In [1]:
from tldraw import TldrawWidgetCoordinates
widget = TldrawWidgetCoordinates()
widget

TldrawWidgetCoordinates()

In [2]:
import bpy
import ipywidgets as widgets
from IPython.display import display

# Function to scale down the coordinates
def scale_down_points(points, scale_factor=0.01):
    scaled_points = [(x * scale_factor, y * scale_factor) for x, y in points]
    return scaled_points

# Function to create the curve
def create_curve_from_points(points):
    # Create a new curve object
    curve_data = bpy.data.curves.new(name='Curve', type='CURVE')
    curve_data.dimensions = '3D'
    curve_data.fill_mode = 'FULL'
    
    # Create a new spline in that curve
    polyline = curve_data.splines.new('POLY')
    polyline.points.add(len(points) - 1)
    
    # Assign points to the spline
    for i, coord in enumerate(points):
        x, y = coord
        polyline.points[i].co = (x, y, 0, 1)  # Z-axis is set to 0, W to 1
    
    # Create an object with the curve data
    curve_obj = bpy.data.objects.new('CurveObject', curve_data)
    
    # Link the object to the main collection
    main_collection = bpy.context.scene.collection
    main_collection.objects.link(curve_obj)
    
    # Apply the Geometry Nodes modifier
    modifier = curve_obj.modifiers.new(name="GeometryNodes", type='NODES')
    modifier.node_group = bpy.data.node_groups["dna_node_setup"]
    
    return curve_obj

# Widget setup
output = widgets.Output()

def on_color_change(change):
    output.clear_output(wait=True)
    with output:
        # Extracting and scaling coordinates
        coord_data = change['new']  # assuming 'coord' is the observed property
        points = [(item['x'], -item['y']) for item in coord_data]
        
        # Scale down points by 0.01
        scaled_points = scale_down_points(points, 0.01)
        
        # Create the curve in Blender
        create_curve_from_points(scaled_points)
        
        # Display the scaled coordinates for verification
        print("Scaled Coordinates:", scaled_points)

# Assuming the widget has been instantiated and the observed property is 'coord'
widget.observe(on_color_change, names='coord')

display(output)

Output()